In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib
from datetime import datetime, timedelta
import copy

df = pd.read_csv("data/dataset2.csv")
df["date"] = pd.to_datetime(df["date"])

model = load_model("model/model.keras", compile=False)
model.compile(optimizer="adam", loss="mse")
scalerX = joblib.load("model/scaler_X.pkl")
scalerY = joblib.load("model/scaler_Y.pkl")
encoder = joblib.load("model/encoder.pkl")

def predict_future_cases(
    problemtype, targetdatestr, df, model, scalerX, scalerY, encoder, sequence_length=30
):
    problemtype_input = problemtype.strip().title()
    try:
        targetdate = pd.Timestamp(targetdatestr)
    except:
        raise ValueError("Invalid date format. Use YYYY-MM-DD.")
    today = pd.Timestamp(datetime.today().date())
    if targetdate <= today:
        raise ValueError("Target date must be in the future from today.")
    if targetdate > today + pd.Timedelta(days=30):
        raise ValueError("Target date exceeds 30-day prediction window.")

    subset = df[df["problem_type"] == problemtype_input].sort_values("date")
    if subset.empty:
        raise ValueError(f"No data found for problem_type '{problemtype_input}'")

    # Prepare sequence, pad if not enough data
    seq = subset.tail(sequence_length).copy().reset_index(drop=True)
    if len(seq) < sequence_length:
        lastrow = seq.iloc[-1]
        for _ in range(sequence_length - len(seq)):
            seq = pd.concat([seq, pd.DataFrame([lastrow])], ignore_index=True)

    lastdate = seq["date"].max()

    while lastdate < targetdate:
        lastseq = seq.tail(sequence_length).copy()
        nextdate = lastdate + pd.Timedelta(days=1)
        newrow = lastseq.iloc[-1].copy()

        # Update dynamic features for realism
        newrow["date"] = nextdate
        newrow["day_of_week"] = nextdate.weekday()
        newrow["is_weekend"] = int(nextdate.weekday() >= 5)
        newrow["month"] = nextdate.month
        newrow["holiday_flag"] = 0  # or your holiday logic
        newrow["prev_day_cases"] = lastseq["reported_cases"].iloc[-1]
        newrow["prev_3day_avg_cases"] = lastseq["reported_cases"].tail(3).mean()

        newrow["severity_score"] = max(
            1, min(4, lastseq["severity_score"].iloc[-1] + np.random.randint(-1, 2))
        )
        newrow["weather_score"] = max(
            0, min(1, lastseq["weather_score"].iloc[-1] + np.random.normal(0, 0.05))
        )
        newrow["rainfall_mm"] = max(
            0, lastseq["rainfall_mm"].iloc[-1] + np.random.normal(0, 2)
        )
        newrow["problem_severity_interaction"] = max(
            1,
            lastseq["problem_severity_interaction"].iloc[-1] + np.random.randint(-1, 2),
        )

        # Prepare model input
        Xseq = lastseq.drop(
            columns=["date", "reported_cases", "workforce_required"]
        )
        cat_encoded = encoder.transform(Xseq[["problem_type", "region"]])
        catdf = pd.DataFrame(cat_encoded, columns=encoder.get_feature_names_out(["problem_type", "region"]))
        Xseq = pd.concat([Xseq.drop(columns=["problem_type", "region"]), catdf], axis=1)
        X_scaled = scalerX.transform(Xseq)
        X_input = np.array([X_scaled])

        y_pred_scaled = model.predict(X_input, verbose=0)
        predictedcases = scalerY.inverse_transform(y_pred_scaled)[0, 0]

        severityfactor = {1: 1, 2: 1.5, 3: 2, 4: 3}
        predictedworkforce = int(predictedcases * severityfactor[int(newrow["severity_score"])])

        newrow["reported_cases"] = predictedcases
        newrow["workforce_required"] = predictedworkforce
        seq = pd.concat([seq, pd.DataFrame([newrow])], ignore_index=True)
        lastdate = nextdate

    # Safely retrieve target row (avoid out-of-bounds errors)
    target_row_df = seq[seq["date"] == targetdate]
    if target_row_df.empty:
        # If exact date not present (rare), take last available row before targetdate
        target_row_df = seq[seq["date"] <= targetdate]
        if target_row_df.empty:
            raise ValueError(f"Prediction failed: target date {targetdate.strftime('%Y-%m-%d')} not in sequence.")
        target_row = target_row_df.iloc[-1]
    else:
        target_row = target_row_df.iloc[-1]

    return {
        "predicted_date": target_row["date"].strftime("%Y-%m-%d"),
        "problem_type": problemtype_input,
        "predicted_cases": round(target_row["reported_cases"], 1),
        "predicted_workforce": int(target_row["workforce_required"])
    }


if __name__ == "__main__":
    problemtype = input("Enter problem type: ") 
    targetdatestr = input("Enter target future date (YYYY-MM-DD): ")  # e.g., "2025-12-31"
    try:
        res = predict_future_cases(
            problemtype,
            targetdatestr,
            df,
            model,
            scalerX,
            scalerY,
            encoder,
            sequence_length=30
        )
        print(res)
    except Exception as e:
        print(f"Error: {e}")


Error: No data found for problem_type ''
